In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

In [2]:
#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#dl libraraies
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense , merge
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ReduceLROnPlateau

from keras.layers.merge import dot
from keras.models import Model

In [3]:
# specifically for deeplearning.
from keras.layers import Dropout, Flatten,Activation,Input,Embedding
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
from IPython.display import SVG

In [4]:
np.random.seed(123)

In [5]:
# Reading the CSV file
train=pd.read_csv('../DATA/ratings_small.csv')

In [6]:
train.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
df_orig = train.copy()

In [8]:
df = df_orig.copy()

In [9]:
# change all the ratings to 1
df['rating']=1

In [10]:
df

,userId,movieId,rating,timestamp
0,1,31,1,1260759144
1,1,1029,1,1260759179
2,1,1061,1,1260759182
3,1,1129,1,1260759185
4,1,1172,1,1260759205
...,...,...,...,...
99999,671,6268,1,1065579370
100000,671,6269,1,1065149201
100001,671,6365,1,1070940363
100002,671,6385,1,1070979663


In [11]:
# fill negative rating
# Get a list of all movie IDs
all_movieIds = df['movieId'].unique()

def fill_negative_rating(df):
# Placeholders that will hold the training data
    users, items, ratings = [], [], []

# This is the set of items that each user has interaction with
    user_item_set = set(zip(df['userId'],df['movieId']))

# 4:1 ratio of negative to positive samples
    num_negatives = 3

    for (u, i) in user_item_set:
        users.append(u)
        items.append(i)
        ratings.append(1) # items that the user has interacted with are positive
        for _ in range(num_negatives):
            # randomly select an item
            negative_item = np.random.choice(all_movieIds) 
        # check that the user has not interacted with this item
            while (u, negative_item) in user_item_set:
                negative_item = np.random.choice(all_movieIds)
            users.append(u)
            items.append(negative_item)
            ratings.append(0) # items not interacted with are negative
    df_new = pd.DataFrame(list(zip(users, items,ratings)),columns =['userId', 'movieId','rating'])
    return df_new

In [12]:
df = fill_negative_rating(df)

In [13]:
df.userId = df.userId.astype('category').cat.codes.values
df.movieId = df.movieId.astype('category').cat.codes.values

users = df.userId.unique()
movies = df.movieId.unique()

userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

df['userId'] = df['userId'].apply(lambda x: userid2idx[x])
df['movieId'] = df['movieId'].apply(lambda x: movieid2idx[x])

In [14]:
x_train,x_test,y_train,y_test=train_test_split(df[['userId','movieId']],df[['rating']],test_size=0.20,random_state=42)

In [15]:
# Creating the Embeddings ,Merging and Making the Model from Embeddings
n_movies=len(df['movieId'].unique())
n_users=len(df['userId'].unique())
n_latent_factors=64  # hyperparamter to deal with

In [16]:
user_input=Input(shape=(1,),name='user_input',dtype='int64')

In [17]:
#user_vec.shape
#user_embedding.shape
user_embedding_mf=Embedding(n_users,n_latent_factors,name='user_embedding')(user_input)
user_vec_mf =Flatten(name='FlattenUsers')(user_embedding_mf)

In [18]:
#movie_vec
movie_input=Input(shape=(1,),name='movie_input',dtype='int64')
movie_embedding_mf=Embedding(n_movies,n_latent_factors,name='movie_embedding')(movie_input)
movie_vec_mf=Flatten(name='FlattenMovies')(movie_embedding_mf)

In [19]:
mf_out=dot([user_vec_mf,movie_vec_mf],name='Simalarity-Dot-Product',axes=1)
# mf_prediction = Dense(1, activation='sigmoid',name = "mf_prediction")(sim)

In [20]:
# user_vec_nn
user_embedding_nn=Embedding(n_users,n_latent_factors,name='nn_user_embedding')(user_input)
user_vec_nn=Flatten(name='nn_FlattenUsers')(user_embedding_nn)
# user_vec=Dropout(0.40)(user_vec)
movie_embedding_nn=Embedding(n_movies,n_latent_factors,name='nn_movie_embedding')(movie_input)
movie_vec_nn=Flatten(name='nn_FlattenMovies')(movie_embedding_nn)
# movie_vec=Dropout(0.40)(movie_vec)
# Concatenate the embeddings (and remove the useless extra dimension)
concatenated = keras.layers.Concatenate()([user_vec_nn, movie_vec_nn])
out1 = keras.layers.Flatten()(concatenated)

# Specifying the Model architecture
nn_inp=Dense(48,activation='relu')(out1)
# nn_inp=Dropout(0.4)(nn_inp)
# nn_inp=BatchNormalization()(nn_inp)
nn_inp=Dense(32,activation='relu')(nn_inp)
nn_inp=Dense(24,activation='relu')(nn_inp)
nn_inp=Dense(16,activation='relu')(nn_inp)
nn_inp=Dense(4,activation='relu')(nn_inp)
# nn_inp=Dropout(0.4)(nn_inp)
nn_out=Dense(1,activation='relu')(nn_inp)

In [21]:
# Concatenate MF and MLP parts
mf_nn_out_concat = keras.layers.Concatenate()([mf_out, nn_out])
# mf_nn_out_concat = keras.layers.Flatten()(mf_nn_out_concat)
# Final prediction layer
mf_nn_prediction = Dense(1, activation='sigmoid',name = "mf_nn_prediction")(mf_nn_out_concat)

In [22]:
combined_model =keras.models.Model([user_input, movie_input],mf_nn_prediction)

In [23]:
combined_model.compile(optimizer=Adam(lr=1e-4),loss='binary_crossentropy',metrics='accuracy')

In [24]:
combined_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
movie_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
nn_user_embedding (Embedding)   (None, 1, 64)        42944       user_input[0][0]                 
__________________________________________________________________________________________________
nn_movie_embedding (Embedding)  (None, 1, 64)        580224      movie_input[0][0]                
_______________________________________________________________________________________

In [25]:
batch_size=128
epochs=20

In [26]:
combined_History = combined_model.fit([x_train.userId,x_train.movieId],y_train.rating, batch_size=batch_size,
                                  epochs =epochs, validation_data = ([x_test.userId,x_test.movieId],y_test.rating),
                                  verbose = 1)

Epoch 1/20
1333/2501 [==============>...............] - ETA: 10s - loss: 0.6775 - accuracy: 0.7438

KeyboardInterrupt: 

In [ ]:
plt.plot(combined_History.history['loss'] , 'g')
plt.plot(combined_History.history['val_loss'] , 'b')
# plt.plot(combined_History.history['val_loss'] , 'b')
plt.title('combined_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.grid(True)
plt.show()

In [ ]:
plt.plot(combined_History.history['accuracy'] , 'g')
plt.plot(combined_History.history['val_accuracy'] , 'b')
# plt.plot(combined_History.history['val_loss'] , 'b')
plt.title('combined_model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show()

In [33]:
# Hit Rate@10
# User-item pairs for testing
x_test1 = x_test[y_test['rating']==1]

test_user_item_set = set(zip(x_test1['userId'], x_test1['movieId']))

# Dict of all items that are interacted with by each user
user_interacted_items = df.groupby('userId')['movieId'].apply(list).to_dict()

# all movie item
all_movieIds = df['movieId'].unique()


hits = []
for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_movieIds) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    
    predicted_ratings = np.squeeze(combined_model.predict([np.array([u]*100),np.array(test_items)]))
    
    top10_items = [test_items[i] for i in np.argsort(predicted_ratings)[::-1][0:10].tolist()]
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 10 is 0.69
